<a href="https://colab.research.google.com/github/xiaoqing16/A-Machine-Learning-Model-to-Diagnose-Depression-using-EEG-Data/blob/main/LOS_Cross_Sell_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [35]:
# import data
%cd /content/drive/My Drive/Colab Notebooks/Data/Cross_Sell
data = pd.read_table("LOS_Cross_Sell_v1.txt", delimiter="|")
data.info()

/content/drive/My Drive/Colab Notebooks/Data/Cross_Sell
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372842 entries, 0 to 372841
Data columns (total 57 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   #customernumber             372842 non-null  int64  
 1   sex                         372842 non-null  object 
 2   calc_customerinfo_age       372842 non-null  int64  
 3   nationality                 372842 non-null  object 
 4   residencystatus             372641 non-null  object 
 5   race                        372842 non-null  int64  
 6   maritalstatus               372842 non-null  int64  
 7   academicqualification       131855 non-null  float64
 8   staywith                    342439 non-null  float64
 9   numberofdependent           372833 non-null  float64
 10  employmenttype              372828 non-null  float64
 11  businessnature              372840 non-null  float64
 12  residentialpostc

In [13]:
# check how many data is null
data.isnull().sum()

#customernumber                    0
sex                                0
calc_customerinfo_age              0
nationality                        0
residencystatus                  201
race                               0
maritalstatus                      0
academicqualification         240987
staywith                       30403
numberofdependent                  9
employmenttype                    14
businessnature                     2
residentialpostcode                2
residentialstatecode               3
residentialcity                    1
employerstatecode                  2
jobtitle                         188
yearofservices                   173
grosssalary                      186
nettsalary                       186
annualincome                     186
paysliptype                      438
totalapplicationcount              0
epcreditlostcount                  0
epactivecount                      0
akpk                               0
fraud                              0
r

In [14]:
# check how many id is unique
uniqueValues = data['#customernumber'].nunique()
print(uniqueValues)

372842


In [36]:
# get percentage count of target variable
data['target'].value_counts()

0    366374
1      6468
Name: target, dtype: int64

In [37]:
# drop all NA for min_cashprice cuz all the target is zero
data = data[data['min_cashprice'].notna()]

# drop this colum because most of the value is NA
data = data.drop(['academicqualification'], axis=1)

# drop gross salary is NA while target is 0 
condition = data['target'].eq(0) & data['grosssalary'].isna()
data = data.loc[~condition]

# drop NA while target is 0
condition = data['target'].eq(0) & data['passscoringflag'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['finalscore'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['currentrepaymentratio'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['newdsr'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['scorerank'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['newrepaymentratio'].isna()
data = data.loc[~condition]

condition = data['target'].eq(0) & data['netdisposableincome'].isna()
data = data.loc[~condition]

# replace inconsistence value
data['residentialstatecode'] = data['residentialstatecode'].replace(['KUALA LUMPUR'],['WP Kuala Lumpur'])
data['employerstatecode'] = data['employerstatecode'].replace(['KUALA LUMPUR'],['WP Kuala Lumpur'])

data.at[207914,'residentialcity']='Sibu'
data.at[207914,'residentialstatecode']='Sarawak'
data.at[207914,'residentialpostcode']='96000'
data.at[207914,'employerstatecode']='Sarawak'

data.at[103021,'residentialcity']='Alor Setar'
data.at[103021,'residentialstatecode']='Kedah'
data.at[103021,'residentialpostcode']='05400'
data.at[103021,'employerstatecode']='WP Kuala Lumpur'

data.at[328413,'residentialstatecode']='Pahang'

data.at[149469,'residentialcity']='Tanah Merah'
data.at[149469,'residentialstatecode']='Kelantan'
data.at[149469,'employerstatecode']='Kelantan'

# replace gen

0    321964
1      2571
Name: target, dtype: int64

In [ ]:
# fill NA by using median of target is 1
condition = data['target'].eq(1)
data_temp = data.loc[condition]

condition = data['target'].eq(0) & data['netdisposableincome'].isna()
data = data.loc[~condition]

data['passscoringflag'] = data['passscoringflag'].fillna(data_temp['passscoringflag'].median())
data['finalscore'] = data['finalscore'].fillna(data_temp['finalscore'].median())
data['currentrepaymentratio'] = data['currentrepaymentratio'].fillna(data_temp['currentrepaymentratio'].median())
data['newdsr'] = data['newdsr'].fillna(data_temp['newdsr'].median())
data['scorerank'].fillna(data_temp['scorerank'].mode()[0])
data['newrepaymentratio'] = data['newrepaymentratio'].fillna(data_temp['newrepaymentratio'].median())
data['netdisposableincome'] = data['netdisposableincome'].fillna(data_temp['netdisposableincome'].median())
 
# fill NA by using median
data[['grosssalary', 'nettsalary', 'annualincome', 'yearofservices']] = data[['grosssalary', 'nettsalary', 'annualincome', 'yearofservices']].fillna(data[['grosssalary', 'nettsalary', 'annualincome', 'yearofservices']].median())

# fill NA by using mode
data['businessnature'].fillna(data['businessnature'].mode()[0])
data['jobtitle'].fillna(data['jobtitle'].mode()[0])
data['residencystatus'].fillna(data['residencystatus'].mode()[0])
data['paysliptype'].fillna(data['paysliptype'].mode()[0])

#fill NA by using "0"
data[['numberofdependent', 'employmenttype']] = data[['numberofdependent', 'employmenttype']].fillna('0')

#fill NA by using "Unknown"
data[['staywith']] = data[['staywith']].fillna('unknown')

# replace inconsistence value
data['residentialstatecode'] = data['residentialstatecode'].replace(['KUALA LUMPUR'],['WP Kuala Lumpur'])
data['employerstatecode'] = data['employerstatecode'].replace(['KUALA LUMPUR'],['WP Kuala Lumpur'])

data.at[207914,'residentialcity']='Sibu'
data.at[207914,'residentialstatecode']='Sarawak'
data.at[207914,'residentialpostcode']='96000'
data.at[207914,'employerstatecode']='Sarawak'

data.at[103021,'residentialcity']='Alor Setar'
data.at[103021,'residentialstatecode']='Kedah'
data.at[103021,'residentialpostcode']='05400'
data.at[103021,'employerstatecode']='WP Kuala Lumpur'

data.at[328413,'residentialstatecode']='Pahang'

data.at[149469,'residentialcity']='Tanah Merah'
data.at[149469,'residentialstatecode']='Kelantan'
data.at[149469,'employerstatecode']='Kelantan'

# replace gen

In [17]:
#fill Na as 0 for 'first_eppfhistory'
data['first_eppfhistory'] =  data['first_eppfhistory'].fillna('9999')
data['first_eppfhistory'] = data['first_eppfhistory'].astype(int)
data.isnull().sum()

#customernumber                  0
sex                              0
calc_customerinfo_age            0
nationality                      0
residencystatus                178
race                             0
maritalstatus                    0
staywith                         0
numberofdependent                0
employmenttype                   0
businessnature                   1
residentialpostcode              0
residentialstatecode             0
residentialcity                  0
employerstatecode                0
jobtitle                        11
yearofservices                   0
grosssalary                      0
nettsalary                       0
annualincome                     0
paysliptype                    233
totalapplicationcount            0
epcreditlostcount                0
epactivecount                    0
akpk                             0
fraud                            0
riskgroup                        0
productlimitoutstanding          0
eppflimitoutstanding

In [18]:
# Define function to take largest digit from a number
def Largestdigit(n):
   largest_digit = 0
   smallest_digit = 9
   while (n):
       digit = n % 10
       # largest digit
       largest_digit = max(digit, largest_digit)
       n = n // 10
   return largest_digit

In [19]:
data['first_eppfhistory'] = data['first_eppfhistory'].apply(Largestdigit)

In [20]:
data['target'].value_counts(normalize=True)

0    0.980348
1    0.019652
Name: target, dtype: float64

In [21]:
#from google.colab import files
#data.to_csv('output.csv', encoding = 'utf-8-sig') 
#files.download('output.csv')